In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def make_dataloader(n, batch_size):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=batch_size)
    return data_loader

In [5]:
import torch.nn as nn
from metal.mmtl.utils.metrics import acc_f1, pearson_spearman
from metal.mmtl.task import ClassificationTask
from metal.mmtl.scorer import Scorer

BATCHSIZE = 32

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo_data = make_dataloader(5000, batch_size=BATCHSIZE)
bar_data = make_dataloader(2000, batch_size=BATCHSIZE)


# custom_metrics = {
#     pearson_spearman: ["pearson_corr", "spearman_corr", "pearson_spearman"]
# }
# scorer = Scorer(["accuracy"], custom_metric_funcs=custom_metrics)

foo = ClassificationTask("foo_task", {"train": foo_data, "valid": foo_data, "test": foo_data}, foo_input, foo_head)
bar = ClassificationTask("bar_task", {"train": bar_data, "valid": bar_data, "test": bar_data}, bar_input, bar_head)
# baz = Task("baz_task", "baz_head", [make_dataloader(100), None, None])
tasks = [foo, bar]

In [ ]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks, device=-1, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model, 
    tasks, 
    n_epochs=3, 
    lr=0.01, 
    progress_bar=False,
    log_unit="epochs",
    log_every=0.1,
    score_every=1.0,
    lr_scheduler="linear",
    warmup_steps = 1,
    warmup_unit = "epochs",
    min_lr = 0.0,
#     patience=10,
#     task_metrics=["foo_task/valid/acc_f1"],
#     trainer_metrics=["glue"],
    test_split="test",
    grad_clip=0.01,
    writer="tensorboard",
    checkpoint=True,
    checkpoint_best=True,
    checkpoint_metric="foo_task/valid/accuracy",
    checkpoint_metric_mode="max",
)

Beginning train loop.
Expecting a total of approximately 5600 examples and 175 batches per epoch from 2 tasks.
[0.10 epo]: TRAIN:[model/loss=7.21e-02, model/lr=9.14e-04]
[0.21 epo]: TRAIN:[model/loss=7.53e-02, model/lr=1.94e-03]
[0.31 epo]: TRAIN:[model/loss=7.91e-02, model/lr=2.97e-03]
[0.41 epo]: TRAIN:[model/loss=6.46e-02, model/lr=4.00e-03]
[0.51 epo]: TRAIN:[model/loss=6.88e-02, model/lr=5.03e-03]
[0.62 epo]: TRAIN:[model/loss=4.89e-02, model/lr=6.06e-03]
[0.72 epo]: TRAIN:[model/loss=4.93e-02, model/lr=7.09e-03]
[0.82 epo]: TRAIN:[model/loss=4.37e-02, model/lr=8.11e-03]
[0.93 epo]: TRAIN:[model/loss=4.08e-02, model/lr=9.14e-03]
{'foo_task': tensor([[4.6060e-01, 5.3940e-01],
        [9.9999e-01, 8.8649e-06],
        [1.8828e-01, 8.1172e-01],
        [3.1915e-03, 9.9681e-01],
        [1.0000e+00, 1.1940e-06],
        [2.2030e-08, 1.0000e+00],
        [1.0000e+00, 3.8695e-12],
        [1.0000e+00, 2.3399e-15],
        [8.4381e-01, 1.5619e-01],
        [6.1579e-02, 9.3842e-01],
     

In [ ]:
loss = 0.0000123456789012345
print(f"{loss:0.3e}")

In [ ]:
# for batch in foo.data_loaders["train"]:
#     X, Y = batch
#     print(model(X, ['foo_task']))
#     print(model.calculate_loss(X, Y, ['foo_task']))    
#     print(model.calculate_output(X, ['foo_task']))    
#     break

In [ ]:
model.predict_probs(foo, "train")

In [ ]:
model.predict(foo, "train")

In [ ]:
model.score(foo, "valid", "accuracy")

In [ ]:
a = {"foo": 1, "bar": 2}
list(a.keys())

In [ ]:
# import os
# filepath = os.path.join(os.environ["METALHOME"], "my_model.pkl")
# model.save(filepath)

In [ ]:
a = []
a[0]